How to start a webdriver client in node?

In [1]:
var importer = require('../Core');
var webdriverio = require('webdriverio');
var sync = require('wdio-sync')
var getSessions = importer.import('load webdriver sessions');

var TIMEOUT = 10000;
var sessionSync = false;

var client;
function createWebdriverClient(host, port) {
    if(sessionSync) {
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => createWebdriverClient(host, port));
    }
    sessionSync = true;
    const sessions = getSessions();
    
    var webdriverServer = {
        sync: true,
        debug: true,
        host: host || 'localhost',
        port: port || 4444,
        logLevel: 'command',
        baseUrl: 'https://webdriver.io',
        pageLoadStrategy: 'eager',
        connectionRetryTimeout: TIMEOUT,
        desiredCapabilities: {
            browserName: 'chrome',
            chromeOptions: {
                prefs: {
                    'download.default_directory': '/data/downloads',
                    'profile.default_content_setting_values.notifications': 2,
                    'exited_cleanly': true,
                    'exit_type': 'None'
                },
                args: [
                    // TODO: https://superuser.com/questions/461035/disable-google-chrome-session-restore-functionality
                    'user-data-dir=/tmp/profile-' + sessions.active.length % 5,
                    // 'start-fullscreen',
                    'no-sandbox',
                    'disable-session-crashed-bubble',
                    'disable-infobars',
                    'new-window',
                    'disable-geolocation',
                    'disable-notifications',
                    'show-saved-copy',
                    'silent-debugger-extension-api'
                    //'kiosk'
                ]
            }
        },
    };
    
    console.log('Initializing webdriver on ' + webdriverServer.host);
    client = webdriverio.remote(webdriverServer);
    const {
        updateOrAddSession,
        getInactiveSessions,
        getActiveSessions
    } = importer.import('manage webdriver sessions', {client});
    client.on('error', e => { console.log(e); this.endAll() });
    client.on('end', () => console.log('Daemon: Closing browser'));
    client.on('result', (result) => updateOrAddSession(client.requestHandler.sessionID));
    
    return client
        .getInactiveSessions()
        // save current session
        .then(validSessions => {
            console.log(validSessions);
            if (validSessions.length == 0) {
                // save new session
                client.requestHandler.sessionID = null;
                // TODO: fix this, doesn't work on second init, keeps opening new windows
                return client.end().catch(e => {}).init().session();
            } else {
                // set to first valid
                client.requestHandler.sessionID = validSessions[0];
                return client.session();
            }
        })
        .then(r => {
            updateOrAddSession(client.requestHandler.sessionID);
            console.log(client.requestHandler.sessionID);
        })
        .catch(e => console.log(e))
        // Down here at the bottom for safetey
        .then(() => (sessionSync = false))
};
module.exports = createWebdriverClient;
createWebdriverClient;


[Function: createWebdriverClient]

Load webdriver sessions?


In [ ]:
var fs = require('fs');
var path = require('path');

var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

var sessions = {};
var sessionModified = 0;

function getSessions() {
    try {
        if(fs.existsSync(SESSIONS_PATH)
           && fs.statSync(SESSIONS_PATH).mtime.getTime() > sessionModified) {
            sessionModified = fs.statSync(SESSIONS_PATH).mtime.getTime();
            sessions = JSON.parse(fs.readFileSync(SESSIONS_PATH)
                .toString());
        }
    } catch (e) {
        sessions = {};
    }

    if(typeof sessions.inactive === 'undefined') {
        sessions.inactive = [];
    }
    if(typeof sessions.active === 'undefined') {
        sessions.active = [];
    }
    return sessions;
};
module.exports = getSessions;


Manage webdriver sessions?



In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getSessions = importer.import('load webdriver sessions');

var TIMEOUT = 10000;
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

function updateOrAddSession(currentSession) {
    const sessions = getSessions();
    const updateSession = sessions.active.filter(s => s[1] === currentSession)[0];
    if(typeof updateSession !== 'undefined') {
        if((new Date()).getTime() - updateSession[0] > TIMEOUT / 2) {
            updateSession[0] = (new Date()).getTime();
        } else {
            return;
        }
    } else {
        sessions.active.push([(new Date()).getTime(), currentSession]);
    }
    fs.writeFileSync(
        SESSIONS_PATH,
        JSON.stringify(sessions, null, 4))
}

function getActiveSessions() {
    const sessions = getSessions();
    const original = client.requestHandler.sessionID;
    const active = [].concat(sessions.active);
    active.sort((a, b) => b[0] - a[0]);
    return importer.runAllPromises(active.map(session => (resolve) => {
        client.requestHandler.sessionID = session[1];
        client
            .windowHandle()
            .then(r => client.window(r.value))
            .session()
            .then(s => resolve(s.sessionId))
            .catch(e => {
                // if the session is really old and has an error delete it from the list
                const index = sessions.active.map(s => s[1]).indexOf(sessions[1]);
                if((new Date()).getTime() - session[0] > 10 * TIMEOUT) {
                    sessions.active.splice(index, 1);
                }
                sessions.active.slice(index, 1);
                resolve(null);
            });
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            updateOrAddSession(original);
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

function getInactiveSessions() {
    const sessions = getSessions();
    const original = client.requestHandler.sessionID;
    // validate and close each session
    const inactive = sessions.inactive.concat(sessions.active.filter(session => 
            // reuse if lagging longer than 120 seconds?
            (new Date()).getTime() - session[0] > TIMEOUT))
        .filter(session => typeof session[1] !== 'undefined' && session[1] !== null)
    inactive.sort((a, b) => a[0] - b[0]);
    return importer.runAllPromises(inactive.map(session => (resolve) => {
        client.requestHandler.sessionID = session[1];
        console.log(session)
        client
            .windowHandle()
            .then(r => client.window(r.value))
            .session()
            .then(s => resolve(s.sessionId))
            .catch(e => resolve(null));
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

if(typeof client.getInactiveSessions === 'undefined') {
    client.addCommand('getInactiveSessions', getInactiveSessions);
}
if(typeof client.getActiveSessions === 'undefined') {
    client.addCommand('getActiveSessions', getActiveSessions);
}
module.exports = {
    updateOrAddSession,
    getInactiveSessions,
    getActiveSessions
}



In [ ]:
$$.async();
var client = createWebdriverClient('localhost', 4444)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost


In [ ]:
$$.async();
client.windowHandles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


How to end the webdriver service?

In [ ]:
client.endAll();



TODO: add decorated logging with screenshots of buttons results can be used: https://github.com/megamindbrian/bots/blob/master/bots/server.js

TODO: transfer state and cache to client

